# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
data = pd.read_csv("cities.csv")

data

,city,lat,lng,temp,humidity,cloudiness,windspeed,country,date
0,Barrow,71.29,-156.79,35.60,74,90,17.22,US,1600807802
1,Srednekolymsk,67.45,153.68,39.45,92,100,2.55,RU,1600807837
2,Svetlaya,46.54,138.33,55.51,82,83,6.82,RU,1600808005
3,Bako,5.78,36.57,70.29,60,71,2.26,ET,1600808005
4,Sioux Lookout,50.10,-91.92,71.60,49,40,6.93,CA,1600807787
...,...,...,...,...,...,...,...,...,...
575,Xaltianguis,17.07,-99.83,87.80,70,5,8.05,MX,1600808128
576,Road Town,18.42,-64.62,83.34,87,100,1.01,VG,1600808129
577,Mason City,43.15,-93.20,80.55,42,1,9.17,US,1600808129
578,San Carlos,9.66,-68.58,85.86,70,91,1.83,VE,1600808129


In [8]:
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph
# Zero cloudiness
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal
# Note: Feel free to adjust to your specifications but be sure to limit the number of 
#      rows returned by your API requests to a reasonable number.

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [10]:
# Heatmap of temperature
locations = data[["lat", "lng"]]
humi = data["humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
humi_layer = gmaps.heatmap_layer(locations, weights=humi)
fig.add_layer(humi_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
hotel_df = data.loc[(data["temp"] <= 80) & (data["windspeed"] <=10) & (data["cloudiness"] == 0)]
hotel_df

,city,lat,lng,temp,humidity,cloudiness,windspeed,country,date
20,Yumen,40.28,97.20,49.96,47,0,1.99,CN,1600807841
23,Port Alfred,-33.59,26.89,58.23,75,0,5.75,ZA,1600808009
35,Port Hedland,-20.32,118.57,66.20,24,0,6.93,AU,1600807843
42,Noshiro,40.20,140.02,61.66,80,0,8.72,JP,1600808012
54,Port Macquarie,-31.43,152.92,52.45,97,0,1.57,AU,1600807879
...,...,...,...,...,...,...,...,...,...
527,Tupiza,-21.45,-65.72,67.28,19,0,7.85,BO,1600808117
550,Broken Hill,-31.95,141.43,50.00,71,0,8.05,AU,1600808124
552,Sevsk,52.15,34.49,54.27,70,0,6.62,RU,1600808124
553,Plettenberg Bay,-34.05,23.37,55.67,69,0,5.03,ZA,1600808124


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df['Hotel_Name'] = ""

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
 # find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get loc from df
    lat = row['lat']
    lng = row['lng']

    # get loc for each row
    params['location'] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['city']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {row['city']} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel_Name'] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 20: Yumen.
Missing field/result... skipping.
------------
Retrieving Results for Index 23: Port Alfred.
Closest Port Alfred hotel is The Halyards Hotel.
------------
Retrieving Results for Index 35: Port Hedland.


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest Port Hedland hotel is The Esplanade Hotel.
------------
Retrieving Results for Index 42: Noshiro.
Closest Noshiro hotel is Azuma Ryokan.
------------
Retrieving Results for Index 54: Port Macquarie.
Closest Port Macquarie hotel is Rydges Port Macquarie.
------------
Retrieving Results for Index 60: Udachny.
Closest Udachny hotel is Severnoye Siyaniye.
------------
Retrieving Results for Index 61: Nyurba.
Closest Nyurba hotel is Gostinitsa Alrosa.
------------
Retrieving Results for Index 64: Carnarvon.
Closest Carnarvon hotel is Hospitality Carnarvon.
------------
Retrieving Results for Index 80: Włoszczowa.
Closest Włoszczowa hotel is Hotel Mariage.
------------
Retrieving Results for Index 90: Albany.
Missing field/result... skipping.
------------
Retrieving Results for Index 97: Airai.
Closest Airai hotel is Juvinal Martin.
------------
Retrieving Results for Index 107: General Roca.
Closest General Roca hotel is Hotel El Recreo.
------------
Retrieving Results for Index 111

In [18]:
hotel_df

,city,lat,lng,temp,humidity,cloudiness,windspeed,country,date,Hotel_Name
20,Yumen,40.28,97.20,49.96,47,0,1.99,CN,1600807841,
23,Port Alfred,-33.59,26.89,58.23,75,0,5.75,ZA,1600808009,The Halyards Hotel
35,Port Hedland,-20.32,118.57,66.20,24,0,6.93,AU,1600807843,The Esplanade Hotel
42,Noshiro,40.20,140.02,61.66,80,0,8.72,JP,1600808012,Azuma Ryokan
54,Port Macquarie,-31.43,152.92,52.45,97,0,1.57,AU,1600807879,Rydges Port Macquarie
...,...,...,...,...,...,...,...,...,...,...
527,Tupiza,-21.45,-65.72,67.28,19,0,7.85,BO,1600808117,La Torre Hotel
550,Broken Hill,-31.95,141.43,50.00,71,0,8.05,AU,1600808124,Daydream Motel & Cottages
552,Sevsk,52.15,34.49,54.27,70,0,6.62,RU,1600808124,Skarabey
553,Plettenberg Bay,-34.05,23.37,55.67,69,0,5.03,ZA,1600808124,Thanda Vista B&B


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [21]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))